In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, welch
from utils import cut_bvp
from constants import Timestamps, expressive


In [2]:
base_path_video = "BVPs"

failed_masks = [
    [2, "Q1_1"],
    [52, "Q7_2"],
    [53, "Q4_2"]
]

paths = [
    "Q1_1",
    "Q1_2",
    "Q2_1",
    "Q2_2",
    "Q3_1",
    "Q3_2",
    "Q4_1",
    "Q4_2",
    "Q5_1",
    "Q5_2",
    "Q6_1",
    "Q6_2",
    "Q7_1",
    "Q7_2",
    "Q8_1",
    "Q8_2",
    "Q9_1",
    "Q9_2"
]

patients = list(range(1, 62))
patients.remove(23)

#patients = expressive

In [3]:
class BVP:
    def __init__(self, patient, path, signal, features):
        self.patient = patient
        self.path = path
        self.signal = signal
        self.features = features

In [4]:
fs = 60

BVPs = []

for patient in patients:

    for path in paths:

        if [patient, path] in failed_masks:
            print(f"Skipping Patient_{patient}, {path}")
            continue

        data = np.load(f"{base_path_video}/Patient_{patient}/{path}.npy")

        #t_start, t_end = getattr(Timestamps, path)

        #data_cut = cut_bvp(data, t_start, t_end, fs)

        #bvp = BVP(patient, path, data_cut, [])

        bvp = BVP(patient, path, data, [])

        BVPs.append(bvp)

        #print(f"Patient_{patient}, {path}: {data.shape}")

print(f"Loaded {len(BVPs)} BVP signals")

Skipping Patient_2, Q1_1
Skipping Patient_52, Q7_2
Skipping Patient_53, Q4_2
Loaded 1077 BVP signals


In [9]:
import numpy as np
from scipy.signal import find_peaks, czt
from scipy.stats import linregress
from itertools import permutations


def compute_bvp_arousal_features(bvp, fs,
                                 fmin=0.7,
                                 fmax=4.0,
                                 n_czt_bins=512):
    """
    Compute arousal-related features from short POS-based BVP signals
    using Chirp Z-Transform (CZT) for spectral analysis.

    Parameters
    ----------
    bvp : np.ndarray
        1D Blood Volume Pulse signal
    fs : float
        Sampling frequency (Hz)
    fmin, fmax : float
        Frequency band of interest (Hz)
    n_czt_bins : int
        Number of CZT frequency bins

    Returns
    -------
    features : dict
        Dictionary of extracted features
    """

    features = {}
    bvp = np.asarray(bvp, dtype=float)

    if len(bvp) < fs * 5:
        # Too short to be meaningful
        for k in [
            "mean_hr", "hr_slope", "rmssd", "ibi_cv",
            "dom_freq", "peak_power_ratio", "spec_entropy",
            "freq_variance", "hr_snr",
            "amp_mean", "amp_std",
            "sample_entropy", "perm_entropy",
            "peak_success_ratio"
        ]:
            features[k] = np.nan
        return features

    # Remove DC
    bvp -= np.mean(bvp)

    # --------------------------------------------------
    # 1. Peak detection & IBI
    # --------------------------------------------------
    min_dist = int(0.4 * fs)  # ~150 bpm upper bound
    peaks, _ = find_peaks(bvp, distance=min_dist)

    if len(peaks) < 3:
        for k in features:
            features[k] = np.nan
        return features

    ibi = np.diff(peaks) / fs
    hr = 60.0 / ibi

    # --------------------------------------------------
    # 2. HR / IBI features
    # --------------------------------------------------
    features["mean_hr"] = np.mean(hr)

    t = np.arange(len(hr))
    features["hr_slope"] = linregress(t, hr).slope

    diff_ibi = np.diff(ibi)
    features["rmssd"] = (
        np.sqrt(np.mean(diff_ibi ** 2)) if len(diff_ibi) > 0 else np.nan
    )

    features["ibi_cv"] = np.std(ibi) / np.mean(ibi)

    # --------------------------------------------------
    # 3. CZT-based spectral features
    # --------------------------------------------------
    N = len(bvp)

    w = np.exp(-1j * 2 * np.pi * (fmax - fmin) / (n_czt_bins * fs))
    a = np.exp(1j * 2 * np.pi * fmin / fs)

    spectrum = czt(bvp, n_czt_bins, w, a)
    power = np.abs(spectrum) ** 2
    freqs = np.linspace(fmin, fmax, n_czt_bins)

    total_power = np.sum(power)

    if total_power > 0:
        idx_peak = np.argmax(power)
        dom_freq = freqs[idx_peak]

        features["dom_freq"] = dom_freq
        features["peak_power_ratio"] = power[idx_peak] / total_power

        p_norm = power / total_power
        features["spec_entropy"] = -np.sum(
            p_norm * np.log2(p_norm + 1e-12)
        )

        features["freq_variance"] = np.sum(
            power * (freqs - dom_freq) ** 2
        ) / total_power

        features["hr_snr"] = np.max(power) / (np.mean(power) + 1e-12)

    else:
        features["dom_freq"] = np.nan
        features["peak_power_ratio"] = np.nan
        features["spec_entropy"] = np.nan
        features["freq_variance"] = np.nan
        features["hr_snr"] = np.nan

    # --------------------------------------------------
    # 4. Pulse amplitude features
    # --------------------------------------------------
    troughs, _ = find_peaks(-bvp, distance=min_dist)
    n_beats = min(len(peaks), len(troughs))

    if n_beats > 0:
        amp = bvp[peaks[:n_beats]] - bvp[troughs[:n_beats]]
        features["amp_mean"] = np.mean(amp)
        features["amp_std"] = np.std(amp)
    else:
        features["amp_mean"] = np.nan
        features["amp_std"] = np.nan

    # --------------------------------------------------
    # 5. Sample entropy
    # --------------------------------------------------
    def sample_entropy(x, m=2, r=0.2):
        x = np.asarray(x)
        r *= np.std(x)
        N = len(x)

        def _phi(m):
            x_m = np.array([x[i:i + m] for i in range(N - m)])
            C = np.sum(
                np.max(
                    np.abs(x_m[:, None] - x_m[None, :]), axis=2
                ) <= r,
                axis=0
            ) - 1
            return np.sum(C) / ((N - m) * (N - m - 1))

        return -np.log(_phi(m + 1) / _phi(m))

    try:
        features["sample_entropy"] = sample_entropy(bvp)
    except Exception:
        features["sample_entropy"] = np.nan

    # --------------------------------------------------
    # 6. Permutation entropy
    # --------------------------------------------------
    def permutation_entropy(x, order=3, delay=1):
        x = np.asarray(x)
        perms = list(permutations(range(order)))
        counts = np.zeros(len(perms))

        for i in range(len(x) - delay * (order - 1)):
            pattern = x[i:i + delay * order:delay]
            idx = perms.index(tuple(np.argsort(pattern)))
            counts[idx] += 1

        p = counts / np.sum(counts)
        return -np.sum(p * np.log2(p + 1e-12))

    try:
        features["perm_entropy"] = permutation_entropy(bvp)
    except Exception:
        features["perm_entropy"] = np.nan

    # --------------------------------------------------
    # 7. Signal quality
    # --------------------------------------------------
    expected_beats = len(bvp) / fs * (features["mean_hr"] / 60.0)
    features["peak_success_ratio"] = (
        len(peaks) / expected_beats if expected_beats > 0 else np.nan
    )

    return features


In [10]:
import numpy as np


def compute_bvp_arousal_features_windowed(
    bvp,
    fs,
    window_sec=5.0,
    fmin=0.7,
    fmax=4.0,
    n_czt_bins=256
):
    """
    Compute arousal features using 5s windows and temporal aggregation.

    Parameters
    ----------
    bvp : np.ndarray
        1D BVP signal
    fs : float
        Sampling rate (Hz)
    window_sec : float
        Window length in seconds (default: 5s)
    fmin, fmax : float
        CZT frequency band
    n_czt_bins : int
        Number of CZT bins

    Returns
    -------
    features : dict
        Aggregated window-based features
    """

    bvp = np.asarray(bvp, dtype=float)
    win_len = int(window_sec * fs)

    if len(bvp) < 2 * win_len:
        # Need at least 2 windows to compute dynamics
        return None

    # --------------------------------------------------
    # 1. Window the signal
    # --------------------------------------------------
    windows = []

    # Regular windows
    for start in range(0, len(bvp) - win_len + 1, win_len):
        windows.append(bvp[start:start + win_len])

    # Force final window (end-aligned)
    end_start = len(bvp) - win_len
    if end_start > 0 and end_start not in range(0, len(bvp) - win_len + 1, win_len):
        windows.append(bvp[end_start:end_start + win_len])


    # --------------------------------------------------
    # 2. Extract features per window
    # --------------------------------------------------
    window_features = []

    for w in windows:
        feats = compute_bvp_arousal_features(
            w,
            fs,
            fmin=fmin,
            fmax=fmax,
            n_czt_bins=n_czt_bins
        )

        if any(np.isnan(v) for v in feats.values()):
            continue

        window_features.append(feats)

    if len(window_features) < 2:
        return None

    # --------------------------------------------------
    # 3. Aggregate across windows
    # --------------------------------------------------
    feature_names = list(window_features[0].keys())
    features = {}

    for name in feature_names:
        values = np.array([wf[name] for wf in window_features])

        features[f"{name}_mean"] = np.mean(values)
        features[f"{name}_std"] = np.std(values)
        features[f"{name}_range"] = np.max(values) - np.min(values)

    return features


In [11]:
fs = 60

valid = []

failed = []

failed_masks = [
    [2, "Q1_1"],
    [52, "Q7_2"],
    [53, "Q4_2"]
]

bvp = BVPs[0]
try:
    
    feats = compute_bvp_arousal_features(bvp.signal, fs)

    if feats is None or []:
        # Do nothing
        print(f"Failed: Patient_{bvp.patient}, {bvp.path}")
        BVPs.remove(bvp)

    else:

        bvp.features = feats

        valid.append(f"Patient_{bvp.patient}, {bvp.path}")

except Exception as e:
    print(e)

print(f"Extracted features for {len(valid)} videos")

print(feats)


Extracted features for 1 videos
{'mean_hr': np.float64(92.67175147884798), 'hr_slope': np.float64(-0.18722620300888415), 'rmssd': np.float64(0.21440089070590135), 'ibi_cv': np.float64(0.26113481903988517), 'dom_freq': np.float64(1.1585127201565557), 'peak_power_ratio': np.float64(0.04793326263115801), 'spec_entropy': np.float64(6.674535467902542), 'freq_variance': np.float64(0.26779362618550123), 'hr_snr': np.float64(24.541830467152685), 'amp_mean': np.float64(0.24989149000154537), 'amp_std': np.float64(0.11648765336562995), 'sample_entropy': np.float64(0.47257829173557264), 'perm_entropy': np.float64(1.4628079769958475), 'peak_success_ratio': np.float64(0.9343615492425701)}


In [12]:
# fs = 60

# valid = []

# failed = []

# failed_masks = [
#     [2, "Q1_1"],
#     [52, "Q7_2"],
#     [53, "Q4_2"]
# ]

# for bvp in BVPs:
    
#     feats = compute_bvp_arousal_features(bvp.signal, fs)

#     if feats is None or []:
#         # Do nothing
#         print(f"Failed: Patient_{bvp.patient}, {bvp.path}")
#         BVPs.remove(bvp)
#         continue
    
#     bvp.features = feats

#     valid.append(f"Patient_{bvp.patient}, {bvp.path}")

#     print(f"Computing Features for Patient_{bvp.patient}...", end="\r", flush=True)

# print(f"Extracted features for {len(valid)} videos")


In [13]:

bvp = BVPs[0].signal
fs = 60
window_sec=5.0
fmin=0.7
fmax=4.0
n_czt_bins=256

bvp = np.asarray(bvp, dtype=float)
win_len = int(window_sec * fs)

if len(bvp) < 2 * win_len:
    # Need at least 2 windows to compute dynamics
    print("Corto")

# --------------------------------------------------
# 1. Window the signal
# --------------------------------------------------
windows = []
for start in range(0, len(bvp) - win_len + 1, win_len):
    windows.append(bvp[start:start + win_len])

# --------------------------------------------------
# 2. Extract features per window
# --------------------------------------------------
window_features = []

for w in windows:
    feats = compute_bvp_arousal_features(
        w,
        fs,
        fmin=fmin,
        fmax=fmax,
        n_czt_bins=n_czt_bins
    )

    print(feats)

    if any(np.isnan(v) for v in feats.values()):
        continue

    window_features.append(feats)

if len(window_features) < 2:
    print("Corto 2")

# --------------------------------------------------
# 3. Aggregate across windows
# --------------------------------------------------
feature_names = list(window_features[0].keys())
features = {}

for name in feature_names:
    values = np.array([wf[name] for wf in window_features])

    features[f"{name}_mean"] = np.mean(values)
    features[f"{name}_std"] = np.std(values)
    features[f"{name}_range"] = np.max(values) - np.min(values)



{'mean_hr': np.float64(91.57073996587674), 'hr_slope': np.float64(13.447182951655376), 'rmssd': np.float64(0.1651598552244999), 'ibi_cv': np.float64(0.25769410160110373), 'dom_freq': np.float64(1.1917647058823528), 'peak_power_ratio': np.float64(0.04851526572513631), 'spec_entropy': np.float64(5.750314487165185), 'freq_variance': np.float64(0.22786962101018568), 'hr_snr': np.float64(12.419908025634241), 'amp_mean': np.float64(0.21433886755371365), 'amp_std': np.float64(0.1234632379308801), 'sample_entropy': np.float64(0.379341089192185), 'perm_entropy': np.float64(1.4857321427592236), 'peak_success_ratio': np.float64(0.9173235908249957)}
{'mean_hr': np.float64(91.70452711896274), 'hr_slope': np.float64(5.4832171265791185), 'rmssd': np.float64(0.27284509239574833), 'ibi_cv': np.float64(0.2587577369012374), 'dom_freq': np.float64(1.1658823529411764), 'peak_power_ratio': np.float64(0.046579963590697854), 'spec_entropy': np.float64(6.028060052994548), 'freq_variance': np.float64(0.26546100

In [14]:
fs = 60  # sampling rate

valid = []
failed = []

failed_masks = [
    [2, "Q1_1"],
    [52, "Q7_2"],
    [53, "Q4_2"]
]

for bvp in BVPs:  # use a copy to safely remove items

    print(f"Computing Features for Patient_{bvp.patient}...", end="\r", flush=True)
    
    try:
        # Use the new windowed feature extraction
        feats = compute_bvp_arousal_features_windowed(
            bvp.signal,
            fs,
            window_sec=5,   # 5-second windows
            fmin=0.7,
            fmax=4.0,
            n_czt_bins=256
        )

        if feats is None or feats == []:
            print(f"Failed: Patient_{bvp.patient}, {bvp.path}")
            failed.append(f"Patient_{bvp.patient}, {bvp.path}")
            BVPs.remove(bvp)  # remove problematic signal
        else:
            bvp.features = feats
            valid.append(f"Patient_{bvp.patient}, {bvp.path}")

    except Exception as e:
        print(f"Error for Patient_{bvp.patient}, {bvp.path}: {e}")
        failed.append(f"Patient_{bvp.patient}, {bvp.path}")
        BVPs.remove(bvp)

print(f"Extracted features for {len(valid)} videos")

# Example inspection
if valid:
    print(list(BVPs[0].features.items()))


Failed: Patient_14, Q1_2atient_14...
Extracted features for 1075 videos..
[('mean_hr_mean', np.float64(91.50525440795484)), ('mean_hr_std', np.float64(5.648315089994841)), ('mean_hr_range', np.float64(17.475377863424427)), ('hr_slope_mean', np.float64(-0.8817931434975907)), ('hr_slope_std', np.float64(10.404024892717763)), ('hr_slope_range', np.float64(27.409008051495)), ('rmssd_mean', np.float64(0.1780005429081016)), ('rmssd_std', np.float64(0.05085446976216987)), ('rmssd_range', np.float64(0.15150992757440637)), ('ibi_cv_mean', np.float64(0.2358113573073545)), ('ibi_cv_std', np.float64(0.07565909027610697)), ('ibi_cv_range', np.float64(0.22879186603497192)), ('dom_freq_mean', np.float64(1.2124705882352942)), ('dom_freq_std', np.float64(0.07393478843667901)), ('dom_freq_range', np.float64(0.20705882352941196)), ('peak_power_ratio_mean', np.float64(0.04594800037319385)), ('peak_power_ratio_std', np.float64(0.004815259826769002)), ('peak_power_ratio_range', np.float64(0.0130960119150919

In [30]:
print("Example of features: ", BVPs[0].features.keys())
print("Features per video: ", len(BVPs[0].features))

Example of features:  dict_keys(['mean_hr_mean', 'mean_hr_std', 'mean_hr_range', 'hr_slope_mean', 'hr_slope_std', 'hr_slope_range', 'rmssd_mean', 'rmssd_std', 'rmssd_range', 'ibi_cv_mean', 'ibi_cv_std', 'ibi_cv_range', 'dom_freq_mean', 'dom_freq_std', 'dom_freq_range', 'peak_power_ratio_mean', 'peak_power_ratio_std', 'peak_power_ratio_range', 'spec_entropy_mean', 'spec_entropy_std', 'spec_entropy_range', 'freq_variance_mean', 'freq_variance_std', 'freq_variance_range', 'hr_snr_mean', 'hr_snr_std', 'hr_snr_range', 'amp_mean_mean', 'amp_mean_std', 'amp_mean_range', 'amp_std_mean', 'amp_std_std', 'amp_std_range', 'sample_entropy_mean', 'sample_entropy_std', 'sample_entropy_range', 'perm_entropy_mean', 'perm_entropy_std', 'perm_entropy_range', 'peak_success_ratio_mean', 'peak_success_ratio_std', 'peak_success_ratio_range'])
Features per video:  42


In [16]:
label_map = {
    "Q1": 1,
    "Q2": 1,
    "Q3": 1,
    "Q4": 0,
    "Q5": 0, 
    "Q6": 0,
    "Q7": -1,
    "Q8": -1,
    "Q9": -1
}

def get_label(path):
    q = path.split("_")[0]  # "Q3_2" → "Q3"
    return label_map[q]



In [17]:
for bvp in BVPs:

    try: 

        for feat, val in bvp.features.items():
            #print(feat, val)
            i = 1
        #print(bvp.patient, bvp.path, "Success")

    except Exception as e:
        print(bvp.patient, bvp.path, e)


14 Q2_1 'list' object has no attribute 'items'


In [18]:
# import numpy as np

# X = []
# y = []

# for bvp in BVPs:
#     if bvp is None or bvp.features is []:
#         print("Error: Patient", bvp.patient, bvp.path)
#         continue

#     feat_values = list(bvp.features.values())
#     X.append(feat_values)
#     y.append(get_label(bvp.path))
    

# X = np.array(X)
# y = np.array(y)

# print(X.shape, y.shape)
# print(np.unique(y, return_counts=True))
# print("Example of data: ", X[0])


LEARNING

In [19]:
# # Random Split

#from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y,
#     test_size=0.2,
#     stratify=y,      # important for emotions
#     random_state=42
# )

# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)

# print("Example of data: ", X_train[0])


In [20]:
# Expressive for test


X_train = []
y_train = []

X_test = []
y_test = []

for bvp in BVPs:
    if bvp is None or bvp.features == []:
        print("Fatal Error: Patient", bvp.patient, bvp.path)
        continue

    elif bvp.patient in expressive:
        #print(bvp.patient, bvp.path, "EXPRESSIVE")
        X_train.append(list(bvp.features.values()))
        y_train.append(get_label(bvp.path))

    else:
        #print(bvp.patient, bvp.path, bvp.features, "REST")
        X_test.append(list(bvp.features.values()))
        y_test.append(get_label(bvp.path))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print("Example of data: ", X_train[0])

Fatal Error: Patient 14 Q2_1
(359, 42) (359,)
(716, 42) (716,)
Example of data:  [ 9.15052544e+01  5.64831509e+00  1.74753779e+01 -8.81793143e-01
  1.04040249e+01  2.74090081e+01  1.78000543e-01  5.08544698e-02
  1.51509928e-01  2.35811357e-01  7.56590903e-02  2.28791866e-01
  1.21247059e+00  7.39347884e-02  2.07058824e-01  4.59480004e-02
  4.81525983e-03  1.30960119e-02  5.97566390e+00  2.36738031e-01
  6.62909289e-01  2.88901087e-01  1.12332265e-01  3.11294110e-01
  1.17626881e+01  1.23270652e+00  3.35257905e+00  2.32094929e-01
  1.68633287e-02  4.30286918e-02  1.14285643e-01  1.83723024e-02
  4.29103141e-02  4.31767417e-01  2.95183580e-02  8.11440484e-02
  1.45239107e+00  2.42066160e-02  7.06100313e-02  9.71515950e-01
  4.97100791e-02  1.13196682e-01]


In [21]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test  = scaler.transform(X_test)


In [22]:
# from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression(
#     max_iter=1000,
#     class_weight="balanced"  # helpful for imbalance
# )

# clf.fit(X_train, y_train)


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced"
    ))
])



In [24]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

X = []
y = []
groups = []

for bvp in BVPs:
    if bvp is None or bvp.features == []:
        print(f"Skipping Patient_{bvp.patient} {bvp.path}")
        continue

    X.append(list(bvp.features.values()))
    y.append(get_label(bvp.path))
    groups.append(bvp.patient)

X = np.array(X)
y = np.array(y)
groups = np.array(groups)

Skipping Patient_14 Q2_1


In [25]:
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score

cv = StratifiedGroupKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scores = cross_val_score(
    pipe,
    X,
    y,
    groups=groups,
    cv=cv,
    scoring="f1_macro"
)

print("F1 macro:", scores.mean(), "±", scores.std())


F1 macro: 0.3465972162899769 ± 0.03286079669171155


In [26]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=200, random_state=42))])

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.32      0.44      0.37       239
           0       0.34      0.25      0.29       240
           1       0.35      0.32      0.33       237

    accuracy                           0.34       716
   macro avg       0.34      0.34      0.33       716
weighted avg       0.34      0.34      0.33       716

[[104  69  66]
 [107  61  72]
 [114  48  75]]


In [28]:
import pandas as pd

# feature names from BVP objects (windowed)
feature_names = list(BVPs[0].features.keys())

# extract RF importances from pipeline
importances = pipe.named_steps['clf'].feature_importances_

# build DataFrame
imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

print(imp_df)


                     feature  importance
3              hr_slope_mean    0.031814
33       sample_entropy_mean    0.028914
28              amp_mean_std    0.028495
41  peak_success_ratio_range    0.028158
39   peak_success_ratio_mean    0.026883
37          perm_entropy_std    0.026684
13              dom_freq_std    0.026410
29            amp_mean_range    0.026230
35      sample_entropy_range    0.026036
12             dom_freq_mean    0.025758
34        sample_entropy_std    0.025647
2              mean_hr_range    0.025402
0               mean_hr_mean    0.025013
40    peak_success_ratio_std    0.024717
25                hr_snr_std    0.024573
38        perm_entropy_range    0.024455
8                rmssd_range    0.024446
20        spec_entropy_range    0.024005
11              ibi_cv_range    0.023899
30              amp_std_mean    0.023745
16      peak_power_ratio_std    0.023675
23       freq_variance_range    0.023396
36         perm_entropy_mean    0.023207
1               